# `scippr` inference module

This notebook outlines the `scippr` inference procedure based on hierarchical inference.  To improve performance, all probabilities will be calculated as log probabilities.

In [ ]:
import numpy as np
import astropy.cosmology as cosmology
import scipy.optimize as spo
import scipy.stats as sps
from scipy.stats import norm
import scipy.linalg as la
import emcee
import corner
from datetime import datetime
import hickle
import bisect
import daft
import cProfile
import StringIO
import pstats
import sys
epsilon = sys.float_info.min
log_epsilon = sys.float_info.min_exp

import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
from matplotlib import rc
rc("font", family="serif", size=12)
rc("text", usetex=True)
colors = 'rbgcymk'

`scippr` is based on a probabilistic graphical model, illustrated below.  The model has two types of observables, shown in shaded circles, supernova lightcurves $\underline{\ell}_{n}$ and host galaxy photometry $\vec{m}_{n}$.  The parameters, which are by definition not directly observable, are shown in empty circles.  The latent variables of supernova type $t_{n}$, redshift $z_{n}$, and distance modulus $\mu_{n}$ are parameters over which we will marginalize, without ever directly inferring them, and while all three of them influence $\underline{\ell}_{n}$, only $z_{n}$ affects $\vec{m}_{n}$ in this model.  In other words, _we currently assume no relationship between supernova type and host galaxy photometry, an assumption we may revisit in the future_.  The selection functions parametrized by $\vec{P}$, $\vec{V}$, $\vec{C}$, and $\vec{M}$ are known constants of the survey symbolized by dots that influence the possible lightcurves and host galaxy photometry that are included in the sample.  The box indicates that the latent variables and the observables are generated independently $N$ times for each supernova in the sample.  The hyperparameters we would like to estimate are the redshift-dependent supernova type proportions $\underline{\phi}$ that determine $t_{n}$ and $z_{n}$ and the cosmological parameters $\vec{\theta}$ that relate $z_{n}$ to $\mu_{n}$, which are shared by all $N$ supernovae in the observed sample.  

In [ ]:
#initialize the PGM
pgm = daft.PGM([6, 4.5], origin=[0, 0])

#desired hyperparameters
pgm.add_node(daft.Node("cosmology", r"$\vec{\theta}$", 2., 4.))
pgm.add_node(daft.Node("dist", r"$\underline{\phi}$", 3.5, 4.))
#pgm.add_node(daft.Node("rates", r"$\vec{R}$", 3., 5.5, fixed=True))

#latent variables/parameters
pgm.add_node(daft.Node("distance", r"$\mu_{n}$", 2., 2.5))
pgm.add_node(daft.Node("redshift", r"$z_{n}$", 3., 3.))
pgm.add_node(daft.Node("type", r"$t_{n}$", 4., 2.5))

#data
pgm.add_node(daft.Node("lightcurve", r"$\underline{\ell}_{n}$", 2.5, 1., observed=True))
pgm.add_node(daft.Node("photometry", r"$\vec{m}_{n}$", 3.5, 1., observed=True))

#known constant parameters
pgm.add_node(daft.Node("lightcurve selection", r"$\vec{P}, \vec{V}, \vec{C}$", 1., 1.75, fixed=True))
pgm.add_node(daft.Node("photometry selection", r"$\vec{M}$", 5., 1.75, fixed=True))

# Add in the edges.
pgm.add_edge("dist", "type")
pgm.add_edge("cosmology", "distance")
pgm.add_edge("dist", "redshift")
pgm.add_edge("redshift", "distance")
#pgm.add_edge("distance", "photometry")
pgm.add_edge("distance", "lightcurve")
pgm.add_edge("redshift", "photometry")
pgm.add_edge("redshift", "lightcurve")
pgm.add_edge("type", "lightcurve")
pgm.add_edge("photometry selection", "photometry")
pgm.add_edge("lightcurve selection", "lightcurve")

# plates
pgm.add_plate(daft.Plate([1.5, 0.5, 3., 3.], label=r"$n = 1, \cdots, N$"))

# Render and save.
pgm.render()
pgm.figure.show()

The probabilistic graphical model will guide us in characterizing the posterior distribution $\ln[p(\vec{\theta}, \underline{\phi} | \{\underline{\ell}_{n}, \vec{m}_{n}\}_{N}, \vec{P}, \vec{V}, \vec{C}, \vec{M})]$ of the hyperparameters given the observed data, but as a sneak preview, this is the form it will take:

\begin{align}
\ln[p(\vec{\theta}, \underline{\phi} | \{\underline{\ell}_{n}, \vec{m}_{n}\}_{N}, \vec{P}, \vec{V}, \vec{C}, \vec{M})] \propto & \ln[p(\vec{\theta}, \underline{\phi})]\\
& +\sum_{n}^{N}\ \ln[\iiint\ \exp[\ln[p(t_{n}, z_{n}, \mu_{n} | \underline{\ell}_{n}, \vec{m}_{n}, \underline{\xi}, \vec{P}, \vec{V}, \vec{C}, \vec{M})]+\ln[p(t_{n}, z_{n}, \mu_{n} | \vec{\theta}, \underline{\phi})]\\
\ \ \ \ \ & -\ln[p(t_{n}, z_{n}, \mu_{n} | \underline{\xi})]]\ d\mu_{n}\ dz_{n}\ dt_{n}]
\end{align}

## Setting up the parameter space

First we set up the parameter space for the latent variables of supernova type $t$, redshift $z$, and distance modulus $\mu$.

In [ ]:
with open('../data.hkl', 'r+') as in_file:
    sim_info = hickle.load(in_file)
    
types = sim_info['types']
n_types = len(types)

z_bins = sim_info['z_bins']
z_difs = z_bins[1:] - z_bins[:-1]
z_mids = (z_bins[1:] + z_bins[:-1]) / 2.
n_zs = len(z_difs)

mu_bins = sim_info['mu_bins']
mu_difs = mu_bins[1:] - mu_bins[:-1]
mu_mids = (mu_bins[1:] + mu_bins[:-1]) / 2.
n_mus = len(mu_difs)

def safe_log(arr, threshold=epsilon):
    shape = np.shape(arr)
    flat = arr.flatten()
    logged = np.log(np.array([max(a, threshold) for a in flat])).reshape(shape)
    return logged

def reg_vals(arr, threshold=log_epsilon):
    arr[arr < threshold] = threshold
    return arr

## Introducing the log-interim posteriors and interim hyperparameters

`scippr` requires inputs in the form of catalogs $\{\ln[p(t_{n}, z_{n}, \mu_{n} | \underline{\ell}_{n}, \vec{m}_{n}, \underline{\xi}, \vec{P}, \vec{V}, \vec{C}, \vec{M})]\}_{N}$ of interim log-posteriors expressed as `3D` arrays constituting probabilities over $t_{n}$, $z_{n}$, and $\mu_{n}$, enabling rapid computation of the log-posterior $\ln[p(\underline{\phi}, \vec{\theta} | \{\underline{\ell}_{n}, \vec{m}_{n}\}_{N}, \vec{P}, \vec{V}, \vec{C}, \vec{M})$ over the hyperparameters $\underline{\phi}$ and $\vec{\theta}$ of scientific interest. 

In [ ]:
lninterimposteriors = sim_info['interim ln posteriors']
(n_SNe, n_types, n_zs, n_mus) = np.shape(lninterimposteriors)

# these are going to get a lot narrower
fig = plt.figure(figsize=(n_types * len(colors), n_SNe * len(colors)))
p = 0
for s in range(n_SNe)[:len(colors)]:
    for t in range(n_types):
        p += 1
        plt.subplot(n_SNe, n_types, p)
        plt.pcolormesh(z_mids, mu_mids, lninterimposteriors[s][t].T, cmap='viridis')#, vmin = 0., vmax = 3.)
        plt.colorbar()
        plt.xlabel(r'$z$')
        plt.ylabel(r'$\mu$')
        plt.axis([z_bins[0], z_bins[-1], mu_bins[0], mu_bins[-1]])

The interim posteriors must always come with interim hyperparameters $\underline{\xi}$ specifying $p(t, z, \mu)$ used to produce them.

In [ ]:
interim_ln_prior = sim_info['interim ln prior']
assert np.isclose(np.sum(np.exp(interim_ln_prior) * z_difs[np.newaxis, :, np.newaxis] * mu_difs[np.newaxis, np.newaxis, :]), 1.)

fig = plt.figure(figsize=(n_types*len(colors), len(colors)))
for t in range(n_types):
    plt.subplot(1, n_types, t+1)
    plt.pcolormesh(z_mids, mu_mids, interim_ln_prior[t].T, cmap='viridis')#, vmin = 0., vmax = 3.)
    plt.title('SN '+types[t]+' log interim prior distribution')
    plt.xlabel(r'$z$')
    plt.ylabel(r'$\mu$')
    plt.axis([z_bins[0], z_bins[-1], mu_bins[0], mu_bins[-1]])
    plt.colorbar()

## Choosing the log-hyperprior probability distribution

As in any Bayesian inference, we must choose a hyperprior distribution over the hyperparameters $\vec{\theta}$ and $\underline{\phi}$ that we wish to estimate.  At this stage we will only attempt to infer $\vec{\theta}=(H_{0}, \Omega_{m0})$.  When we evaluate the log-hyperprior probability $\ln[p(\vec{\theta}, \underline{\phi})]$, we will then be evaluating the log-probability of the hyperprior distribution at the given values of the hyperparameters $\vec{\theta}$ and $\underline{\phi}$.  Thus, the log-hyperprior probability is a scalar as expected.

In [ ]:
def lnprior(hyperparameters):
    cosmo_hyperparameters = hyperparameters#['theta']
#     dist_hyperparameters = hyperparameters['phi']
    cosmo_prior_prob = prior_cosmo_dist.logpdf(cosmo_hyperparameters)
#     dist_prior_prob = prior_n_of_z.logpdf(dist_hyperparameters.flatten())
    return cosmo_prior_prob # + dist_prior_prob

We will make the assumption that $H_{0}$ and $\Omega_{m0}$ are independent, i.e. $p(\vec{\theta}) = p(H_{0})\ p(\Omega_{m0})$.

In [ ]:
class indie_dist(object):
    def __init__(self, in_dists):
        self.dists = in_dists
        self.n_dists = len(self.dists)
    def rvs(self, n_samps = 1):
        samps = []
        for d in range(self.n_dists):
            samps.append(self.dists[d].rvs(n_samps))
        return np.array(samps).T
    def logpdf(self, locs):
        n_items = len(locs)
        lnprob = 0
        for d in range(self.n_dists):
            lnprob += self.dists[d].logpdf(locs[d])
        return lnprob

We make the choice to set hyperprior distributions to be Gaussian for $H_{0}$ and truncated normal for $\Omega_{m0}$ (so it must be a fraction) with means equal to the WMAP values for these parameters and standard deviations equal to the $3\sigma$ errors on those measurements.

In [ ]:
n_sigmas = 2.

# WMAP, with 10 * errors so we can see what's going on in crappy plots
wmap_H0 = 70.0
delta_H0 = 2.2 * n_sigmas
[H0_mean, H0_std] = [wmap_H0, delta_H0]
min_H0, max_H0 = 50., 90.
H0_range = max_H0 - min_H0
H0_low, H0_high = (min_H0 - H0_mean) / H0_std, (max_H0 - H0_mean) / H0_std
H0_dist = sps.norm(H0_mean, H0_std)#sps.truncnorm(H0_low, H0_high, loc = H0_mean, scale = H0_std)

wmap_Om0 = 1. - 0.721
delta_Om0 = 0.025 * n_sigmas
[Om0_mean, Om0_std] = [wmap_Om0, delta_Om0]
min_Om0, max_Om0 = 0., 1.
Om0_low, Om0_high = (min_Om0 - Om0_mean) / Om0_std, (max_Om0 - Om0_mean) / Om0_std
Om0_dist = sps.norm(Om0_mean, Om0_std)#sps.truncnorm(Om0_low, Om0_high, loc = Om0_mean, scale = Om0_std)

prior_cosmo_hyperparams = np.array([H0_dist.rvs(), Om0_dist.rvs()])
cosmo_param_names = [r'$H_{0}$', r'$\Omega_{m0}$']
n_cosmo_hyperparams = len(prior_cosmo_hyperparams)
prior_cosmo_dist = indie_dist([H0_dist, Om0_dist])
prior_cosmo = cosmology.FlatLambdaCDM(H0=prior_cosmo_hyperparams[0], Om0=prior_cosmo_hyperparams[1])

def param_check(params):
    if params[0] > min_H0 and params[0] < max_H0 and params[1] > min_Om0 and params[1] < max_Om0:
        return True
    else:
        return False

# # prior on phi is gaussian about flat in log space
# prior_mean = np.ones((n_types, n_zs)) / n_types
# prior_mean /= np.sum(prior_mean * z_difs[np.newaxis, :])
# assert np.isclose(np.sum(prior_mean * z_difs[np.newaxis, :]), 1.)
# prior_mean = safe_log(prior_mean.flatten())
# prior_sigmas = np.array([0.5, 1., 0.25])
# prior_covariance = la.block_diag(prior_sigmas[0] * np.eye(n_zs), prior_sigmas[1] * np.eye(n_zs), prior_sigmas[2] * np.eye(n_zs))
# # prior_n_of_z_hyperparams = [prior_mean, prior_sigmas]
# prior_n_of_z_dist = sps.multivariate_normal(mean = prior_mean, cov = prior_covariance)

H0_grid = 50. + 50. * np.arange(0., 1., 0.01)
Om0_grid = np.arange(0., 1., 0.01)
prior_space = np.zeros((100, 100))
for i in range(100):
    for j in range(100):
        prior_space[i][j] = lnprior(np.array([H0_grid[i], Om0_grid[j]]))
plt.pcolormesh(H0_grid, Om0_grid, prior_space.T, cmap='viridis')#, vmin = 0., vmax = 3.)
plt.colorbar()
plt.xlabel(r'$H_{0}$')
plt.ylabel(r'$\Omega_{m0}$')
plt.axis([H0_grid[0], H0_grid[-1], Om0_grid[0], Om0_grid[-1]])

## Calculating the log-hyperlikelihood

The log-hyperlikelihood $\ln[p(t_{n}, z_{n}, \mu_{n} | \vec{\theta}, \underline{\phi})] = \ln[p(t_{n}, z_{n} | \underline{\phi})]+\ln[p(\mu_{n} | z_{n}, \vec{\theta})]$ is the sum of two terms separable in the hyperparameters.  In our parametrization, the first is derived from a constant lookup table that can be neglected for now and the second is a $\delta$ function located at the `cosmo.distmod()` function evaluated at the given redshift where `cosmo` is defined using the cosmological parameters in $\vec{\theta}$.  We will at this stage need to introduce a function mapping redshifts into distance moduli under a given cosmology.  We note that we will be using some particular values of the log-hyperlikelihood repeatedly so will define them as constants.

In [ ]:
lninterimhyperlikelihood = interim_ln_prior
flat_hyperlikelihood = np.ones(np.shape(interim_ln_prior)) * epsilon
# flat_hyperlikelihood /= np.sum(flat_hyperlikelihood * z_difs[np.newaxis, :, np.newaxis] * mu_difs[np.newaxis, :, np.newaxis])
# assert np.isclose(np.sum(flat_hyperlikelihood * z_difs[np.newaxis, :, np.newaxis] * mu_difs[np.newaxis, :, np.newaxis]), 1.)
flat_lnhyperlikelihood = safe_log(flat_hyperlikelihood)#log_epsilon * np.ones(np.shape(interim_ln_prior))

def lnhyperlikelihood(hyperparameters):
    cosmo_hyperparameters = hyperparameters#['theta']
#     dist_hyperparameters = hyperparameters['phi']
    if param_check(hyperparameters):
        sample_cosmo = cosmology.FlatLambdaCDM(H0=cosmo_hyperparameters[0], Om0=cosmo_hyperparameters[1])
        delta = mu_binner(sample_cosmo.distmod(z_mids).value)
        delta /= np.sum(delta * z_difs[np.newaxis, :, np.newaxis] * mu_difs[np.newaxis, np.newaxis, :] * np.ones((n_types))[:, np.newaxis, np.newaxis] / n_types)
        assert np.isclose(np.sum(delta * z_difs[np.newaxis, :, np.newaxis] * mu_difs[np.newaxis, :, np.newaxis] * np.ones((n_types))[:, np.newaxis, np.newaxis] / n_types), 1.)
        lnprob = safe_log(delta) # + dist_hyperparameters[:, :, np.newaxis]
    else:
        lnprob = flat_lnhyperlikelihood
    return lnprob

def mu_binner(mus):
    matrix = []
    for mu in mus:
        vector = np.zeros(n_mus)
        ind = bisect.bisect(mu_bins[:-1], mu) - 1
        vector[ind] += 1.
        matrix.append(vector)
    return np.array([matrix] * n_types)

## Constructing the log-posterior probability

The full log-posterior probability takes the following form:
\begin{equation}
\ln[p(\vec{\theta}, \underline{\phi} | \{\underline{\ell}_{n}, \vec{m}_{n}\}_{N})] \propto \ln[p(\vec{\theta}, \underline{\phi})]+\sum_{n}^{N}\ \ln[\iiint\ \exp[\ln[p(t_{n}, z_{n}, \mu_{n} | \underline{\ell}_{n}, \vec{m}_{n}, \underline{\xi})]+\ln[p(t_{n}, z_{n}, \mu_{n} | \vec{\theta}, \underline{\phi})]-\ln[p(t_{n}, z_{n}, \mu_{n} | \underline{\xi})]]\ d\mu_{n}\ dz_{n}\ dt_{n}]
\end{equation}
In words, that's the sum of the log-prior probability and the sum of the logs of the integrals over the sum of the log-interim posteriors, the log-hyperlikelihood, and the negative log-interim hyperlikelihood.

In [ ]:
const_term = reg_vals(lninterimposteriors - lninterimhyperlikelihood[np.newaxis, :])
assert(np.all(np.exp(const_term) * z_difs[np.newaxis, :, np.newaxis] * mu_difs[np.newaxis, np.newaxis, :] / n_types) <= 0.)

def lnposterior(hyperparameters):
    new_term = lnhyperlikelihood(hyperparameters)[np.newaxis, :]
    in_exp = reg_vals(const_term + new_term)
#     assert np.all(in_exp) <= 0.
    in_log = np.sum(np.sum(np.sum(np.exp(in_exp) * mu_difs[np.newaxis, np.newaxis, :], axis=3) * z_difs[np.newaxis, :], axis=2) / n_types, axis=1)
    in_sum = safe_log(in_log)
    summed = np.sum(in_sum, axis=0)
#     assert summed <= 0.
    return summed

def lnhyperposterior(hyperparameters):
    return max(lnprior(hyperparameters) + lnposterior(hyperparameters), log_epsilon)

# MCMC Sampler

In [ ]:
mcrandstep = 10.**-1.
nthreads = 4
nwalkers = 100
nsteps = 1000
ninit = 100
output_chain = True
emcee_chain_output = "emcee_chain_testing.dat"
gr_limit = 1.2

In [ ]:
def sample(sampler, ivals, n_samps):
    sampler.reset()
    stuff = sampler.run_mcmc(ivals, n_samps)
    mcmc_outputs = {}
    mcmc_outputs['chain'] = sampler.chain
    mcmc_outputs['lnprobability'] = sampler.lnprobability
    mcmc_outputs['acceptance_fraction'] = sampler.acceptance_fraction
#     mcmc_outputs['acor'] = sampler.acor
    return mcmc_outputs

def single_parameter_gr_stat(chain):
    ssq = np.var(chain, axis=1, ddof=1)
    W = np.mean(ssq, axis=0)
    xb = np.mean(chain, axis=1)
    xbb = np.mean(xb, axis=0)
    m = chain.shape[0]
    n = chain.shape[1]
    B = n / (m - 1.) * np.sum((xbb - xb)**2., axis=0)
    var_x = (n - 1.) / n * W + 1. / n * B
    R_hat = np.sqrt(var_x / W)
    return R_hat

def multi_parameter_gr_stat(sample):
    dims = np.shape(sample)
    (n_walkers, n_iterations, n_params) = dims
    n_burn_ins = n_iterations / 2
    chain_ensemble = np.swapaxes(sample, 0, 1)
    chain_ensemble = chain_ensemble[n_burn_ins:, :]
    Rs = np.zeros((n_params))
    for i in range(n_params):
        chains = chain_ensemble[:, :, i].T
        Rs[i] = single_parameter_gr_stat(chains)
    return Rs

def gr_test(sample, threshold=gr_limit):
    gr = multi_parameter_gr_stat(sample)
    print('Gelman-Rubin test statistic = '+str(gr))
    return np.max(gr) > threshold
    
def run_MCMC(ivals):
    sampler = emcee.EnsembleSampler(nwalkers, n_cosmo_hyperparams, lnhyperposterior, threads=nthreads)
    burn_ins = 0
    burning_in = True
    vals = ivals
    full_chain = np.array([[ivals[w]] for w in range(nwalkers)])
    while burning_in:
        print('began sampling '+str(burn_ins))
        burn_in_mcmc_outputs = sample(sampler, vals, ninit)
        print('finished sampling '+str(burn_ins))
        with open('mcmc'+str(burn_ins)+'.hkl', 'w') as file_location:
            hickle.dump(burn_in_mcmc_outputs, file_location)
        progress_plotter(burn_ins)
        full_chain = np.concatenate((full_chain, burn_in_mcmc_outputs['chain']), axis=1)
        burning_in = gr_test(full_chain)
        vals = np.array([item[-1] for item in burn_in_mcmc_outputs['chain']])
        burn_ins += 1
    mcmc_outputs = sample(sampler, vals, nsteps)
    full_chain = np.concatenate((full_chain, mcmc_outputs['chain']), axis=1)
    with open('full_chain.hkl', 'w') as file_location:
        hickle.dump(full_chain, file_location)
    return full_chain
        
def progress_plotter(burn_ins):
    with open('mcmc'+str(burn_ins)+'.hkl', 'r') as file_location:
        to_plot = hickle.load(file_location)
    fig = plt.figure(figsize=(10, 5))
    sps_prob = fig.add_subplot(1, 2, 1)
    for w in range(nwalkers):
        sps_prob.plot(to_plot['lnprobability'][w], alpha=0.1)
    sps_prob.set_title('lnprobability evolution for samples '+str(burn_ins*ninit)+'-'+str((burn_ins+1)*ninit))
    sps_chain = fig.add_subplot(1, 2, 2)
    for w in range(nwalkers):
        sps_chain.scatter(to_plot['chain'][w].T[0], to_plot['chain'][w].T[1], alpha=0.01)
    sps_chain.scatter(67.9, 1. - 0.693, color='r')
    sps_chain.set_xlabel(cosmo_param_names[0])
    sps_chain.set_ylabel(cosmo_param_names[1])
    fig.savefig('progress_'+str(burn_ins)+'.png')
    fig.show()
    return

In [ ]:
init_positions = []
while len(init_positions) < nwalkers:
    pair = prior_cosmo_dist.rvs()[0]
    if param_check(pair):
        init_positions.append(pair)
init_positions = np.array(init_positions)
init_probs = np.array([lnposterior(ival) for ival in init_positions])
plt.hist(init_probs)

# init_positions = prior_cosmo_dist.rvs(nwalkers)
        
# plt.scatter(init_positions.T[0], init_positions.T[1])

pr = cProfile.Profile()
pr.enable()

results = run_MCMC(init_positions)

pr.disable()
s = StringIO.StringIO()
sortby = 'cumtime'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()

# Comparison

In [ ]:
# for w in range(nwalkers):
#     to_plot = results['chain'][w].T
#     plt.scatter(to_plot[0], to_plot[1], alpha=0.01)#1./np.mean(results['probs'][w]))
# plt.scatter(67.9, 1. - 0.693, color='r')
# plt.xlabel(cosmo_param_names[0])
# plt.ylabel(cosmo_param_names[1])
# plt.savefig('bananas.png')

# Comparison

In [ ]:
with open('../truth.hkl', 'r+') as true_file:
    true_info = hickle.load(true_file)   
true_phi = true_info['phi']
true_theta = true_info['theta']

with open('full_chain.hkl', 'r') as file_location:
    mcmc_outputs = hickle.load(file_location)
# figure = corner.corner(mcmc_outputs)
# figure.savefig('corner.png')

In [ ]:
fig = plt.figure(figsize=(n_cosmo_hyperparams * len(colors), len(colors)))
priorvals = [H0_mean, Om0_mean]
for p in range(n_cosmo_hyperparams):
    plt.subplot(1, n_cosmo_hyperparams, p + 1)
    for w in range(nwalkers):
        to_plot = mcmc_outputs[w].T
        plt.plot(to_plot[p], alpha=0.1)#1./np.mean(results['probs'][w]))#, to_plot[1])
    plt.plot([true_theta[p]] * nsteps, c='k', linewidth=2)
    plt.xlabel('sample number')
    plt.ylabel(cosmo_param_names[p])
fig.savefig('evolution.png')

In [ ]:
for w in range(nwalkers):
    plt.scatter(mcmc_outputs[w].T[0], mcmc_outputs[w].T[1], alpha=0.01)
plt.scatter(67.9, 1. - 0.693, color='r')
plt.xlabel(cosmo_param_names[0])
plt.ylabel(cosmo_param_names[1])
plt.savefig('final.png')

In [ ]:
# for_corner = mcmc_outputs.reshape(nwalkers*nsteps, 2)
# figure = corner.corner(for_corner)
# figure.savefig('corner.png')